# Исследование надежности заемщиков банка


## Описание проекта


На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.


Нужно рассмотреть следующие вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

4. Как разные цели кредита влияют на его возврат в срок?

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## Загрузка и обзор данных

In [2]:
# Загрузка библиотек

import pandas as pd

In [3]:
# Чтение данных из файла

data = pd.read_csv('/datasets/data.csv')

In [4]:
# Выведем первые 10 строчек датафрейма

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
# Выведем основную информацию о датафрейме

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
# рассмотрим гендерный состав

data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

## Предобработка данных

### Удаление пропусков

In [26]:
# Выведем количество пропущенных значений для каждого столбца

data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 В двух столбцах есть пропущенные значения. Один из них — `days_employed`, общий трудовой стаж в днях.  Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно заменить медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [40]:
# Замена отсутствующих значений в столбце 'total_income' на медиану

for category in data['income_type'].unique():
    data.loc[(data['income_type'] == category) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == category), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.

In [34]:
# Замена отрицательных значения с столбце 'days_employed' положительными

data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [35]:
# Медианное значение трудового стажа 'days_employed'

data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец скорее всего не понадобится для исследования.

Выведите перечень уникальных значений столбца `children`

In [36]:
# Уникальные значения столбца 'children'

data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [37]:
# Удалим аномальные значения

data = data[(data['children'] != -1) & (data['children'] != 20)]

In [38]:
# Прроверим уникальные значения столбца 'children'

data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [39]:
# Замена отсутствующих значений в столбце 'days_employed' на медиану

for category in data['income_type'].unique():
    data.loc[(data['income_type'] == category) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == category), 'days_employed'].median()

In [41]:
#  Проверим, что все пропуски заполнены

data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [43]:
# Заменим тип данных на int

data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

В столбце в столбце `education` есть неявные дубликаты - это одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверьте остальные столбцы.

In [50]:
# Приведем столбец `education`к нижнему регистру

data['education'] = data['education'].str.lower()

In [53]:
# Выведем количество строк-дубликатов в данных

data.duplicated().sum()

71

In [54]:
# Удалим дубликаты

data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [55]:
# Функция для определения категории дохода

def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [56]:
# Создание нового столбца 'total_income_category' в DataFrame 'data', который будет содержать категории дохода

data['total_income_category'] = data['total_income'].apply(categorize_income)

In [58]:
# Проверка распределения значений в новом столбце 'total_income_category'

data['total_income_category'].value_counts()

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

Выведем перечень уникальных целей взятия кредита из столбца `purpose`

In [59]:
# Уникальные цели взятия кредита из столбца 'purpose'

data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [60]:
# Функция для определения категории заданной строки (row).

def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [61]:
# Создаем новый столбец 'purpose_category'

data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Анализ данных и поиск ответов на поставленные вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [63]:
# Получаем уникальные значения из столбца 'children'

data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [64]:
# Получим уникальные значения для столбца 'debt'

data['debt'].unique()

array([0, 1])

В столбце 'debt' приведена информация о наличии долгов у семьи: 0 - не было долгов, 1 - долги были.

In [65]:
# Получим количество каждого значения для столбца 'children'

data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Данные неравномерные, рассматривается всего 9 семей с пятью детьми и 41 с четырьмя. Будем укрупнять группы. Для дальнейшей работы семьи с тремя и более детьми будут рассматриваться как одна группа 'многодетные'. Эта категория семей, все равно самая малочисленная в выборке и составляет около 2%, поэтому сделать однозначный вывод относительно многодетных семей может быть затруднительно.

##### Получим таблицу для поиска и анализа зависимости

In [77]:
# Функция для определения типа семьи в зависимости от количества детей

def family_type(children):
    if children == 0:
        return 'бездетная'
    elif children == 1:
        return '1 ребенок'
    elif children == 2:
        return '2 ребенка'
    return 'многодетная'

In [81]:
# Применяем функцию family_type к столбцу 'children' и создаем новый столбец 'family_type' с результатами
data['family_type'] = data['children'].apply(family_type)

# Группируем данные по 'family_type' и рассчитываем количество кредитополучателей, количество должников и долю должников
child_debt_table = data.groupby('family_type')['debt'].agg(['count', 'sum'])
child_debt_table['percent'] = round(child_debt_table['sum'] / child_debt_table['count'] * 100, 2)

# Переименовываем столбцы для удобства восприятия
child_debt_table = child_debt_table.rename(columns={'count': 'Количество кредитополучателей', 'sum': 'Количество должников', 'percent': 'Доля должников'})

# Сортируем таблицу по убыванию доли должников
child_debt_table.sort_values(by='Доля должников', ascending=False)

,Количество кредитополучателей,Количество должников,Доля должников
family_type,,,
2 ребенка,2052,194,9.45
1 ребенок,4808,444,9.23
многодетная,380,31,8.16
бездетная,14091,1063,7.54


В полученной таблице наблюдается зависимость между количеством детей и фактом возврата кредитных средств в срок. Семьи без детей реже всего попадают в долги и из предложенных категорий являются самыми перспективными заемщиками. Исходя из  преведенных данных можно сделать вывод, что многодетные семьи также успешно выплачивают кредиты. Однако, это скорее всего ошибочный вывод из-за недостаточной выборки. В пользу этого так же можно привести аргумент, что семьи с одним ребенком чаще имеют долги, чем семьи без детей; семьи с двумя детьми чаще, чем семьи с одним ребенком. Скорее всего ситуация для многодетных семей еще более сложная.

In [82]:
# Функция определения типа семьи в зависимости от количества детей

def family_type(children):
    if children == 0:
        return 'бездетная'
    elif children == 1:
        return '1 ребенок'
    return 'многодетная'

In [83]:
# Создание нового столбца 'family_type' с помощью функции family_type, примененной к столбцу 'children'
data['family_type'] = data['children'].apply(family_type)

# Группировка данных по столбцу 'family_type' и агрегация данных по столбцу 'debt'
child_debt_table = data.groupby('family_type')['debt'].agg(['count', 'sum'])

# Расчет процентного соотношения должников к общему количеству кредитополучателей
child_debt_table['percent'] = round(child_debt_table['sum'] / child_debt_table['count'] * 100, 2)

# Переименование столбцов для ясного отображения информации
child_debt_table = child_debt_table.rename(columns={'count': 'Количество кредитополучателей', 'sum': 'Количество должников', 'percent': 'Доля должников'})

# Сортировка таблицы по убыванию значения 'Доля должников'
child_debt_table.sort_values(by='Доля должников', ascending=False)

,Количество кредитополучателей,Количество должников,Доля должников
family_type,,,
многодетная,2432,225,9.25
1 ребенок,4808,444,9.23
бездетная,14091,1063,7.54


**Вывод:** Была обнаружена зависимость между количеством детей и фактом возврата кредитных средств в срок. Люди, которые не имеют детей, лучше всего отдают кредит, доля должников здесь составляет с 7,5%, хуже обстоит дело с кредиторами, кто имеет больше одного ребенка, здесь доля невозврата составляет соответственно 9,3%. Более детально категории  некорректно сравнивать, так как в них очень маленькая выборка.
Для более детального анализа необходимо расширить и сбалансировать датасет.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Получим уникальные значения для столбца 'family_status' и узнаем их количество

In [84]:
# Уникальные значения для столбца 'family_status'

data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [85]:
# Подсчет количества уникальных значений в столбце 'family_status'

data['family_status'].value_counts()

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

Выборка не очень равномерная, что может привести к неточностям в оценке.

##### Получим таблицу для поиска и анализа зависимости

In [86]:
# Группировка данных по семейному статусу и агрегация информации о задолженности
marriage_debt_table = data.groupby('family_status')['debt'].agg(['count', 'sum'])

# Расчет процентной доли должников от общего количества кредитополучателей
marriage_debt_table['percent'] = round(marriage_debt_table['sum'] / marriage_debt_table['count'] * 100, 2)

# Переименование столбцов для более ясного понимания содержания
marriage_debt_table = marriage_debt_table.rename(columns={'count': 'Количество кредитополучателей', 'sum': 'Количество должников', 'percent': 'Доля должников'})

# Сортировка таблицы по убыванию доли должников
marriage_debt_table.sort_values(by='Доля должников', ascending=False)

,Количество кредитополучателей,Количество должников,Доля должников
family_status,,,
Не женат / не замужем,2796,273,9.76
гражданский брак,4134,385,9.31
женат / замужем,12261,927,7.56
в разводе,1189,84,7.06
вдовец / вдова,951,63,6.62


**Вывод:** Анализ таблицы показывает, что одинокие люди в прошлом состаявшие в браке хорошо справляются с кредитными обязательствами, должников среди них 6,6 - 7 %. Семейные пары так же достаточно ответственно платят по кредитам, должников среди них 7,6 %. Пары проживающие гражданским браком, не женатые и не замужние граждане самые ненадежные, должников среди них 9,3 % и 9,8 % соответственно.

Надо заметить, что категории клиентов не состоящих в браке самые малочисленные в представленной выборке. Вероятно это случайность. Однако, может оказаться так, что одинокие люди просто реже берут кредиты.

Вероятно, наиболее надежные плательщики - это люди имеющие и осознающие ответственность, например, за детей. В дальнейших исследованиях можно выяснить есть ли корреляция. Так же не совсем понятно ведут ли себя одинаково мужчины и женщины. Эту зависимость тоже нужно изучить.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Получим таблицу для поиска и анализа зависимости

In [87]:
# Группируем данные по категориям общего дохода ('total_income_category') и рассчитываем статистику по задолженностям
salary_debt_table = data.groupby('total_income_category')['debt'].agg(['count', 'sum'])

# Рассчитываем процент должников от общего числа кредитополучателей в каждой категории дохода
salary_debt_table['percent'] = round(salary_debt_table['sum'] / salary_debt_table['count'] * 100, 2)

# Переименовываем столбцы для более ясного понимания содержания таблицы
salary_debt_table = salary_debt_table.rename(columns={'count': 'Количество кредитополучателей',
                                                    'sum': 'Количество должников',
                                                    'percent': 'Доля должников'})

# Сортируем таблицу по убыванию значения "Доля должников"
salary_debt_table.sort_values(by='Доля должников', ascending=False)

,Количество кредитополучателей,Количество должников,Доля должников
total_income_category,,,
E,22,2,9.09
C,15921,1353,8.50
A,25,2,8.00
B,5014,354,7.06
D,349,21,6.02


**Вывод:** Данные по категориям А, Е и D не достаточно презентабельны, так как сформированы из очень маленькой выборки.
Клиенты из категории В (богатые люди) реже попадают в долги (7% должников), чем клиенты категории С (средний класс) среди которых 8% должников.

Для дальнейших исследований рекомендуется расширить выборку.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

##### Получим таблицу для поиска и анализа зависимости

In [88]:
# Группируем данные по категории цели кредита ('purpose_category') и агрегируем информацию о задолженности
purpose_debt_table = data.groupby('purpose_category')['debt'].agg(['count', 'sum'])

# Рассчитываем процент должников в каждой категории и добавляем этот показатель в таблицу
purpose_debt_table['percent'] = round(purpose_debt_table['sum'] / purpose_debt_table['count'] * 100, 2)

# Переименовываем столбцы для более ясного понимания содержания таблицы
purpose_debt_table = purpose_debt_table.rename(columns={'count': 'Количество кредитополучателей', 'sum': 'Количество должников', 'percent': 'Доля должников'})

# Сортируем таблицу по убыванию доли должников, чтобы выделить категории с наибольшим процентом должников
purpose_debt_table.sort_values(by='Доля должников', ascending=False)

,Количество кредитополучателей,Количество должников,Доля должников
purpose_category,,,
операции с автомобилем,4279,400,9.35
получение образования,3988,369,9.25
проведение свадьбы,2313,183,7.91
операции с недвижимостью,10751,780,7.26


**Вывод:**  Данные в таблице показывают, что кредиты на разные цели выплачиваются по-разному. Так, клиенты взявшие кредиты на свадьбу или на покупку недвижимости реже попадают в долги (7,9% и 7,3% соответственно), чем клиенты, которые взяли кредит на покупку автомобиля (9,4% должников) или получение образования (9,3% должников).

## Выводы

Было проведено исследование надежности заемщиков для кредитного отдела банка. Изучалось влияние семейного положения и количества детей, уровня дохода и целей кредита клиента на факт погашения кредита в срок.

Изученные входные данные от банка (статистика о платёжеспособности клиентов) оказались неоднородными. Для некоторых рассматриваемых категорий выборка была очень мала, что не позволило провести детальное исследование. Среди данных были обнаружены и удалены пропуски. Были обработаны дубликаты и аномальные значения. Для ответов на поставленные вопросы была проведена категоризация данных.

Подтвердились гипотезы о влиянии изучаемых параметров на факт погашения кредита в срок:

**1.** Люди, которые не имеют детей, лучше всего отдают кредит, доля должников здесь составляет с 7,5%, хуже обстоит дело с кредиторами, кто имеет больше одного ребенка, здесь доля невозврата составляет соответственно 9,3%.

**2.** Одинокие люди в прошлом состаявшие в браке хорошо справляются с кредитными обязательствами, должников среди них 6,6 - 7%. Семейные пары так же достаточно ответственно платят по кредитам, должников среди них 7,6%. Пары проживающие гражданским браком, не женатые и не замужние граждане самые ненадежные, должников среди них 9,3% и 9,8% соответственно.
Надо заметить, что категории клиентов не состоящих в браке самые малочисленные в представленной выборке. Вероятно это случайность. Однако, может оказаться так, что одинокие люди просто реже берут кредиты.

**3.** Клиенты из категории В (богатые люди) реже попадают в долги (7% должников), чем клиенты категории С (средний класс) среди которых 8% должников. С другой стороны количество данных по этой группе клиентов не велико и выводы могут быть не точны. По причине того что количество данных по категориям A (очень богатые), D (бедные) и E (очень бедные) не велико и выводы могут быть не точны эти категории не были рассмотрены.

**4.** Оказалось, что кредиты на разные цели выплачиваются с разным успехом. Данные показывают, что кредиты на разные цели выплачиваются по-разному. Так, клиенты взявшие кредиты на свадьбу или на покупку недвижимости реже попадают в долги (7,9% и 7,3% соответственно), чем клиенты, которые взяли кредит на покупку автомобиля (9,4% должников) или получение образования (9,3% должников).



**Резюме**
- Чаще всего проблемы в выплатой кредитов имеются у клиентов с детьми, не состоящиев браке с доходом  до 50 т.р., которые берут кредит на образование или оперции с автомобилем.
- Самые надежные клиенты состоят или состояли в браке, не имеющие детей с доходом от 50 т.р., которые берут кредит на недвижимость и свадьбы.
    
**Рекомендации**
- Необходимо расширить и сбалансировать датасет. Например, на данный момент в таблице приведены данные 14092 женщин и 7238 мужчин.
- При сборе данных сделать обязательные поля для заполнения, это поможет избежать пропусков в данных.
- В случае увеличения данных в датасете будет хорошо провести повторное исследование в котором будут учтены категории, которые не удолось учесть в настоящем исследовании.
- В дальнейших исследованиях можено дополнительно изучить влияние возраста и наличия образования на факт своевременного погашения кредитов.